In [1]:
%env CUDA_VISIBLE_DEVICES=0,1


env: CUDA_VISIBLE_DEVICES=0,1


In [1]:
import json 

PATH_LOG = "./"

faked_data = {
  "R02-1-A": [
    "The family court may decide to commence an assistance also in respect of a person whose capacity to appreciate their own situation is extremely inadequate due to a mental disorder.",
    [
      [
        "11",
        "Part I General Provisions Chapter II Persons Section 3 Capacity to Act   (Decisions for Commencement of Curatorship) Article 11  The family court may decide to commence a curatorship in respect of a person whose capacity to appreciate their own situation is extremely inadequate due to a mental disorder, at the request of the person in question, the person's spouse, the person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the assistant, the assistant's supervisor, or a public prosecutor; provided, however, that this does not apply to a person in respect of whom a cause set forth in Article 7 exists."
      ],
      [
        "15",
        "Part I General Provisions Chapter II Persons Section 3 Capacity to Act   (Decisions for Commencement of Assistance) Article 15  (1) The family court may decide to commence an assistance in respect of a person whose capacity to appreciate their own situation is inadequate due to a mental disorder, at the request of the person in question, that person's spouse, that person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the curator, the curator's supervisor, or a public prosecutor; provided, however, that this does not apply to a person with respect to whom there are grounds as prescribed in Article 7 or the main clause of Article 11. \n (2) The issuance of a decision for commencement of assistance at the request of a person other than the person in question requires the consent of the person in question. \n (3) A decision for commencement of assistance must be made concurrent with a decision as referred to in Article 17, paragraph (1) or a decision as referred to in Article 876-9, paragraph (1)."
      ]
    ]
  ],
  "R02-1-I": [
    "The issuance of a decision for commencement of assistance at the request of a person other than the person in question requires the consent of the person in question unless the family court finds it to be suitable.",
    [
      [
        "15",
        "Part I General Provisions Chapter II Persons Section 3 Capacity to Act   (Decisions for Commencement of Assistance) Article 15  (1) The family court may decide to commence an assistance in respect of a person whose capacity to appreciate their own situation is inadequate due to a mental disorder, at the request of the person in question, that person's spouse, that person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the curator, the curator's supervisor, or a public prosecutor; provided, however, that this does not apply to a person with respect to whom there are grounds as prescribed in Article 7 or the main clause of Article 11. \n (2) The issuance of a decision for commencement of assistance at the request of a person other than the person in question requires the consent of the person in question. \n (3) A decision for commencement of assistance must be made concurrent with a decision as referred to in Article 17, paragraph (1) or a decision as referred to in Article 876-9, paragraph (1)."
      ]
    ]
  ],
  "R02-1-U": [
    "If the grounds of commencement of assistance cease to exist, the family court may rescind the decision for commencement of assistance without any party's request.",
    [
      [
        "18",
        "Part I General Provisions Chapter II Persons Section 3 Capacity to Act   (Rescission of Decisions for Commencement of Assistance) Article 18  (1) If the grounds prescribed in the main clause of Article 15, paragraph (1) cease to exist, the family court must rescind the decision for commencement of assistance at the request of the person in question, that person's spouse, that person's relative within the fourth degree of kinship, the guardian of a minor, the supervisor of a minor's guardian, the assistant, the assistant's supervisor, or a public prosecutor. \n (2) At the request of a person as prescribed in the preceding paragraph, the family court may rescind all or part of the decision referred to in paragraph (1) of the preceding Article. \n (3) If rescinding the decision referred to in paragraph (1) of the preceding Article and the decision referred to in Article 876-9, paragraph (1) in their entirety, the family court must rescind the decision for commencement of assistance."
      ]
    ]
  ],
  "R02-1-E": [
    "If the assistant does not consent to an act for which the person under assistance must obtain the assistant's consent even though it is unlikely to prejudice the interests of the person under assistance, the family court may grant permission that operates in lieu of the assistant's consent, at the request of the person under assistance.",
    [
      [
        "17",
        "Part I General Provisions Chapter II Persons Section 3 Capacity to Act   (Decisions Requiring Person to Obtain Consent of Assistant) Article 17  (1) At the request of the person referred to in the main clause of Article 15, paragraph (1) or the assistant or assistant's supervisor, the family court may decide that the person under assistance must obtain the consent of the person's assistant in order to perform a specific juridical act; provided, however, that the acts that such a decision may establish as those for which the person must obtain the consent of the assistant are restricted to a part of the acts provided for in Article 13, paragraph (1). \n (2) In order to decide as referred to in the preceding paragraph at the request of a person other than the person in question requires the consent of the person in question. \n (3) If the assistant does not consent to an act for which the person under assistance must obtain the assistant's consent even though it is unlikely to prejudice the interests of the person under assistance, the family court may grant permission that operates in lieu of the assistant's consent, at the request of the person under assistance. \n (4) An act for which the person under assistance must obtain the assistant's consent is voidable if the person performs it without obtaining the assistance's consent or a permission that operates in lieu of it."
      ]
    ]
  ],
  "R02-2-E": [
    "A foreign juridical person permitted possesses the same private rights even if a foreign national is not entitled to enjoy the rights as those possessed by a juridical person of the same kind that has been formed in Japan.",
    [
      [
        "36",
        "Part I General Provisions Chapter III Juridical Persons Section 1 Incorporation of Juridical Persons   (Foreign Juridical Persons) Article 36  (1) With the exception of states, administrative divisions of states, and commercial companies, the formation of foreign juridical persons is not permitted; provided, however, that this does not apply to a foreign juridical persons that is permitted pursuant to the provisions of a law or treaty. \n (2) A foreign juridical person permitted pursuant to the provisions of the preceding paragraph possesses the same private rights as those possessed by a juridical person of the same kind that has been formed in Japan; provided, however, that this does not apply to a right that a foreign national is not entitled to enjoy or to any right for which there are special provisions in a law or treaty."
      ]
    ]
  ]
}

In [4]:
import re
def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def llm_infer(model, tokenizer_, texts):
    inputs = tokenizer_(texts, return_tensors="pt", padding='longest')["input_ids"].cuda()
    outputs = model.generate(inputs, max_new_tokens=256)
    raw_out = tokenizer_.batch_decode(outputs, skip_special_tokens=True)
    output_text = [format_output(e.replace(texts[i], "")) for i, e in enumerate(raw_out)]
    
    return output_text

In [5]:
def reranking_prompting(list_articles, query_content):
    prompting = f"In bellow articles:  "
    for a_id, a_content in list_articles:
        prompting = prompting + f"\nArticle {a_id}: {a_content},"
        
    prompting  = prompting +f"\nQuestion: which articles really relevant to query \"{query_content}\"?"
    return prompting


In [6]:

prompt_texts = []
prompt_texts_ids = []
for q_id, (query_content, info_relevant_articles) in faked_data.items():
    prompt = reranking_prompting(info_relevant_articles, query_content)
    prompt_texts.append(prompt)
    prompt_texts_ids.append( q_id )
 

In [7]:
print(prompt_texts[0])
print(prompt_texts_ids[0])

In bellow articles:  
Article 11: Part I General Provisions Chapter II Persons Section 3 Capacity to Act   (Decisions for Commencement of Curatorship) Article 11  The family court may decide to commence a curatorship in respect of a person whose capacity to appreciate their own situation is extremely inadequate due to a mental disorder, at the request of the person in question, the person's spouse, the person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the assistant, the assistant's supervisor, or a public prosecutor; provided, however, that this does not apply to a person in respect of whom a cause set forth in Article 7 exists.,
Article 15: Part I General Provisions Chapter II Persons Section 3 Capacity to Act   (Decisions for Commencement of Assistance) Article 15  (1) The family court may decide to commence an assistance in respect of a person whose capacity to appreciate their own situation is inadequate due to a mental disorder, at the

In [8]:

from transformers import AutoTokenizer, BloomForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xxl"
model = AutoModelForSeq2SeqLM.from_pretrained(
		model_name, device_map="auto",  torch_dtype=torch.float16, load_in_8bit=True,
	)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/phuongnm/coliee2024/env_coliee/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 5/5 [00:58<00:00, 11.66s/it]


In [9]:
print(llm_infer(model, tokenizer, prompt_texts[:2]))


['article 15', 'article 15']


In [29]:
import json 
from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader


print("len(prompt_texts) = ", len(prompt_texts))
prompting_loader = DataLoader(prompt_texts, batch_size=4, collate_fn=None, shuffle=False)
all_outputs = []
all_inputs = []
for i_batch, batch_prompt_texts in enumerate(tqdm(prompting_loader)):
    batch_outputs = llm_infer(model, tokenizer,  batch_prompt_texts)
    all_inputs = all_inputs + batch_prompt_texts
    all_outputs = all_outputs + batch_outputs
    if i_batch % 5 == 4:
        json.dump(list(zip(prompt_texts_ids[:len(all_outputs)], all_inputs, all_outputs)), open(f"{PATH_LOG}/all_infer_rerank.json", "wt"), indent=2)

        
json.dump(list(zip(prompt_texts_ids[:len(all_outputs)], all_inputs, all_outputs)), open(f"{PATH_LOG}/all_infer_rerank.json", "wt"), indent=2)

len(prompt_texts) =  400


100%|██████████| 100/100 [19:22<00:00, 11.62s/it]
